This notebook will take a directory of patch images and split them as follows
/train/lg
/train/hg
/val/lg
/val/hg

In [1]:
from sklearn.model_selection import train_test_split
from pathlib import Path
import pandas as pd
patch_directory = "/Users/dng/Desktop/WSI Project/FL_Patches"
patch_directory=Path(patch_directory)

base_directory = "/Users/dng/Desktop/WSI Project/FL_Patches_Split_data"
base_directory = Path(base_directory)
folder_dict = {"train_lg": "train/lg/",
               "train_hg": "train/hg/",
                "val_lg": "val/lg/",
                "val_hg": "val/hg"}



In [2]:
for folder in folder_dict.keys():
    to_make=base_directory.joinpath(folder_dict[folder])
    to_make.mkdir(parents=True,exist_ok=True)

In [5]:
unique_files=[]
for f in patch_directory.glob("*.jpg"):
    filename = f.stem.split("_")[0]
    if filename not in unique_files:
        unique_files.append(filename)

print(unique_files)

['SP17-28311', 'HP19-341', 'HP19-294', 'SP17-20213', 'SP17-22549', 'HP19-961', 'SP17-22446']


In [7]:
decoder = {"HP19-294":"HG",
          "HP19-341":"LG",
          'SP17-28311':"LG",
           'SP17-20213':"LG",
           'SP17-22549':"LG",
           'HP19-961':"LG",
           'SP17-22446':"LG" }

In [8]:
db = []
#this will include only files that we can classify!!!!
for f in patch_directory.glob("*.jpg"):
    for name in decoder.keys():
        if name in f.stem:
            to_add = {"path":f,
                     "classification":decoder[name]}
            db.append(to_add)
            break



In [9]:
db = pd.DataFrame(db)

In [11]:
Data_train, Data_val, Label_train, Label_val = train_test_split(db.path, 
                                                                db.classification, 
                                                                test_size=0.15, 
                                                                random_state=64)

In [13]:
db_train = pd.concat([Data_train,Label_train],axis=1)
db_val = pd.concat([Data_val,Label_val],axis=1)
db_train.shape, db_val.shape

((13481, 2), (2380, 2))

In [14]:
import shutil

def copy_files(row,target):
    if row['classification'] is "HG":
        shutil.copy(row['path'],target / "hg")
    elif row['classification'] is "LG":
        shutil.copy(row['path'],target / "lg")
    else: #failsafe condition to find cases that may have gotten in
        shutil.copy(row['path'],target / "unknown")


In [15]:
temp = db_train.apply(copy_files,axis=1,target = base_directory / "train")

In [16]:
temp = db_val.apply(copy_files,axis=1,target = base_directory / "val")

((2337, 2), (1152, 2))

[]